In [1]:
%matplotlib qt
import os
import broadbean as bb
import numpy as np
import pandas as pd
import panel as pn
import tempfile
import qcodes as qc
import matplotlib.pyplot as plt
from sequencebuilder.alazar_config import alazarconfig
from sequencebuilder.refrenceinstrument import AlazarInstrument
from broadbean.plotting import plotter
from qcodes.utils.dataset.doNd import do0d, do1d
from sequencebuilder.spinbuilder import AWGController
from qcodes import initialise_or_create_database_at, \
    load_or_create_experiment, Measurement, Parameter, \
    Station
from sequencebuilder.spinbuilder import SpinBuilder
from pulsequantum.elem_from_plot import elem_on_plot

pn.extension('tabulator')
ramp = bb.PulseAtoms.ramp

In [2]:
db_path = os.path.join(tempfile.gettempdir(),
                       'data_access_example.db')
initialise_or_create_database_at(db_path)

experiment = load_or_create_experiment(
    experiment_name='alazar',
    sample_name='alazar-sample')
SC = qc.Station(config_file="C:\\Users\\Farzad\\Documents\\Rasmus\\QCoDeS-experiment\\ConfigRBC.yaml")
AWGA = SC.load_instrument('AWGA')
#seqbuild = AWGController(name = 'seqbuild', awg=AWGA)


Connected to: TEKTRONIX AWG5014C (serial:B051370, firmware:SCPI:99.0 FW:4.6.0.7) in 0.21s


In [3]:
alazar = SC.load_instrument('alazar')
alazarconfig(alazar, seqmode=True,external_clock=False)
alazar_ctrl = SC.load_instrument('alazar_ctrl')
channelA_oneD = SC.load_instrument('channelA_oneD',parent=alazar_ctrl)
alazar_ctrl.channels.append(channelA_oneD)
seqbuild = AlazarInstrument(name = 'seqbuild', awg=AWGA, data_func=channelA_oneD.data.get, n_points=100)
SC.add_component(seqbuild)

'seqbuild'

In [7]:
seqbuild.ch_x = 1
seqbuild.ch_y = 2
seqbuild.divider = {'ch1': 0.5, 'ch2': 0.5} 
seqbuild.df = pd.read_csv('spin_funnel.csv', index_col=0)

In [8]:
#seqbuild.spinfunnel()


In [9]:
df_widget = pn.widgets.Tabulator(seqbuild.df)
df_widget


Tabulator(value=       name  time  type   ...)

In [10]:
seqbuild.seq_from_df()
seqbuild.seq.plot_elem_nr(1)
seqbuild.seq.plot_elem_nr(200)

I was here
I was here


In [ ]:
x,y,ramp = elem_on_plot(33)

In [ ]:
seqbuild.df_from_list(x,y,ramp)

In [ ]:
seqbuild.update_seq_from_list(x,y,ramp)

In [ ]:
seqbuild.seq.plot_elem_nr(1)

In [11]:
seqbuild.uploadToAWG()

Sequence uploaded in 24.283388137817383 seconds


In [12]:
seqbuild.awg.ch1_state(1)
seqbuild.awg.ch2_state(1)

In [13]:
print(seqbuild.awg.ch1_state())
print(seqbuild.awg.ch2_state())


1
1


In [14]:
seqbuild.awg.run()

'Running'

In [15]:
seqbuild.awg.get_state()

'Running'

In [16]:
alazar_ctrl.int_time(0.9e-6)
alazar_ctrl.int_delay(1e-7)
channelA_oneD.num_averages(300)
channelA_oneD.records_per_buffer(200)
channelA_oneD.prepare_channel()

#channelA_oneD.data.get()

In [17]:
channelA_oneD.data.get()

array([ 3.72977717e-04,  3.70879121e-04,  3.66681929e-04,  3.62293956e-04,
        3.66109585e-04,  3.51800977e-04,  3.57715201e-04,  3.48366911e-04,
        3.47031441e-04,  3.51419414e-04,  3.41689560e-04,  3.50656288e-04,
        3.39781746e-04,  3.42643468e-04,  3.43788156e-04,  3.42643468e-04,
        3.39018620e-04,  3.26999389e-04,  3.49511600e-04,  3.34630647e-04,
        3.37301587e-04,  3.21848291e-04,  3.26427045e-04,  3.26427045e-04,
        3.20322039e-04,  3.24328449e-04,  3.14789377e-04,  3.15934066e-04,
        3.11927656e-04,  3.12500000e-04,  3.16315629e-04,  3.16506410e-04,
        3.16124847e-04,  3.10210623e-04,  3.05822650e-04,  2.83882784e-04,
        2.97237485e-04,  3.07539683e-04,  3.07539683e-04,  3.03151709e-04,
        3.00480769e-04,  2.96665140e-04,  2.82165751e-04,  2.89224664e-04,
        2.90178571e-04,  2.86172161e-04,  2.72245116e-04,  2.76823871e-04,
        2.76442308e-04,  2.83692002e-04,  2.72817460e-04,  2.84645910e-04,
        2.72817460e-04,  

In [18]:
do0d(channelA_oneD.data, do_plot=True)

Starting experimental run with id: 44. 


(results #44@C:\Users\Farzad\AppData\Local\Temp\data_access_example.db
 ---------------------------------------------------------------------
 alazar_ctrl_channelA_oneD_records - array
 alazar_ctrl_channelA_oneD_data - array,
 [<AxesSubplot:title={'center':'Run #44, Experiment alazar (alazar-sample)'}, xlabel='Records', ylabel='mydata (μV)'>],
 [None])

In [ ]:
testbla = AlazarInstrument('bla',channelA_oneD.data.get,100)

In [ ]:
do0d(testbla.datarefrence, do_plot=True)

In [ ]:
do1d(testbla.V_dc, 0, 1, 10, 0.01,testbla.datarefrence, do_plot=True)

In [ ]:
seqbuild.df.to_csv('test.csv')

In [ ]:
seqbuild.df = pd.read_csv('test.csv', index_col=0)

In [ ]:
a = seqbuild.get_recurcive_info('time')

In [ ]:
b = [x for x in a[1:] if x]

In [ ]:
b

In [ ]:
def get_recurcive(seqbuild, dim_name):
    recurcive_info = seqbuild.get_recurcive_info(dim_name)
    return [x for x in recurcive_info if x]
        

def updata_setpoints(seqbuild):
    time_recurcive = get_recurcive(seqbuild,'time')
    
    if len(time_recurcive) == 1:
        time_setpoint =  np.linspace(*time_recurcive[0], endpoint=True)
        return time_setpoint
    
    chx_recurcive = get_recurcive(seqbuild,f'ch{seqbuild.ch_x}')
    chy_recurcive = get_recurcive(seqbuild,f'ch{seqbuild.ch_y}')
    vector_list = list(zip(chx_recurcive[0],chy_recurcive[0]))
    norm_list = [np.sqrt(x[0]**2+x[1]**2) for x in vector_list]
    return norm_list

In [ ]:
updata_setpoints(seqbuild)

In [ ]:
seqbuild.ch_x

In [ ]:
a = get_recurcive(seqbuild,f'ch{seqbuild.ch_x}')

In [ ]:
len(a)

In [ ]:
chx_recurcive = get_recurcive(seqbuild,f'ch{seqbuild.ch_x}')
chy_recurcive = get_recurcive(seqbuild,f'ch{seqbuild.ch_y}')

In [ ]:
a = list(zip(chx_recurcive[0],chy_recurcive[0]))

In [ ]:
chx_recurcive[0]

In [ ]:
norm = [np.sqrt(x[0]**2+x[1]**2) for x in a]

In [ ]:
norm

In [ ]:
seqbuild.updata_setpoints()